In [1]:
# En caso de problemas, utilizar las dependencias de librerías de este requierement: https://github.com/googlecolab/backend-info/blob/d6d345cb94fc5fd49951c9af0f6ead5e962bfab2/pip-freeze.txt
!pip install numpy==1.23.5
!pip install transformers[torch]==4.35.2
!pip install accelerate -U
!pip install evaluate

  Using cached transformers-4.35.2-py3-none-any.whl (7.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 4.7 MB/s eta 0:00:00
  Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
def load_prepare_data(path):
  """
  Función para cargar y procesar datos para el ejercicio.
  """
  df = pd.read_csv(path,sep=",")
  map_classes = {
    "religion":1,
    "age":1,
    "ethnicity":1,
    "gender":1,
    "other_cyberbullying":1,
    "not_cyberbullying":0,
  }
  df["cyberbullying"] = df.cyberbullying_type.map(map_classes)
  return df[["tweet_text","cyberbullying"]].copy()

# Ejercicio


En este ejercicio vamos a trabajar con un conjunto de datos procedente de medios sociales online.

Uno de los mayores problemas en el internet de hoy en día es la presencia de actitudes negativas hacia algunos colectivos en relación a su etnia, género, religión o ideología política. En este ejercicio trabajaremos con un conjunto de datos reales, etiquetados manualmente, procedentes de la plataforma [Kaggle](https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification/data). Originalmente, a cada documento del dataset se le asignó una de las siguientes categorías:
- *religion*
- *age*
- *ethnicity*
- *gender*
- *other_cyberbullying*
- *not_cyberbullying*


El objetivo inicial del dataset era su uso para entrenar un modelo capaz de detectar el tipo de contenido de odio presente en internet según el colectivo al que se atacaba. En este caso, para simplificar el ejercicio, se ha generado una función `load_prepare_data()` que cambia las categorías del dataset obteníendose al final 2 categorías con valor 1 o 0, indicando si el tweet tiene contenido de odio

**En este ejercicio debeis entrenar un modelo de clasificación utilizando la librería Transformers.** Dado que el análisis exploratorio ha sido realizado en el ejercicio anterior, en este caso podréis centraros en entrenar el modelo utilizando la librería Transformers, seleccionando un modelo pre-entrenado adecuado, entrenando el modelo y llevando a cabo la evaluación.


**Nota 1**: Este ejercicio requiere el uso de las GPUs de Google Colab. Este Colab debería estar preconfigurado para ejecutarse en GPU, pero si tuviera problemas en la ejecución que me contacte a través del Moodle para buscar soluciones alternativas.

## 0. Imports


In [3]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   AutoModelForSequenceClassification,
   AdamW
)
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import sys
from tqdm.autonotebook import tqdm
tqdm.pandas()

import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import classification_report

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## 1. Obtención del corpus
Para la obtención de los datos teneis disponible la función `load_prepare_data()`. Esta función prepara los datos del ejercicio en formato Pandas dataframe para que podais realizarlo.

In [28]:
path_data = "https://raw.githubusercontent.com/luisgasco/ntic_master_datos/main/datasets/cyberbullying_tweets.csv"
# Path de datos alternativos en caso de que el anterior no funcione (al estar alojado en github puede haber limitaciones
# en la descarga.
# path_data = "https://zenodo.org/records/10938455/files/cyberbullying_tweets.csv?download=1"
dataset = load_prepare_data(path_data)

In [29]:
dataset.head(4)

,tweet_text,cyberbullying
0,"In other words #katandandre, your food was cra...",0
1,Why is #aussietv so white? #MKR #theblock #ImA...,0
2,@XochitlSuckkks a classy whore? Or more red ve...,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",0


## 2. Análisis exploratorio

Podéis saltarlo en este ejercicio.

## 3. Preprocesado y Normalización

In [30]:
texts = dataset.tweet_text.values  # an array of strings
labels = dataset.cyberbullying.values  # an array of integers

Como trabajamos con la librería Transformers, tenemos que ajustar nuestros datos al formato de entrada esperado por el modelo.

Dividimos el conjunto de datos en Train-Validation(90%) y Test(10%)

In [31]:
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.25, random_state=0,
                                                    stratify = labels)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=0,stratify = train_labels)

Al trabajar con la librería Transformers, es importante ajustar nuestros datos al formato de entrada esperado por el modelo. Cada modelo preentrenado utiliza un tokenizador específico diseñado para dividir los textos de manera óptima para su entrada. En este caso, utilizaremos el modelo cardiffnlp/twitter-roberta-base-2019-90m

In [32]:
model_name = "cardiffnlp/twitter-roberta-base-2019-90m"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Comprobamos con el tokenizador subdivide un texto de ejemplo:

In [33]:
texto = "Cats and zebras love to nap in sunny spots"
texto_tokens = tokenizer(texto).tokens()
texto_tokens

['<s>',
 'C',
 'ats',
 'Ġand',
 'Ġze',
 'br',
 'as',
 'Ġlove',
 'Ġto',
 'Ġnap',
 'Ġin',
 'Ġsunny',
 'Ġspots',
 '</s>']

Para poder trabajar con Hugging Face es necesario cargar los datos, siguiendo la estructura basada en Apache Arrow.

En este caso, utilzaremos la función CustomDataset(), que tomará la entrada de los textos y etiquetas de cada subset y se prepara el formato requerido para el entrenamiento, realizando la tokenización internamente.

In [34]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        """
        Constructor de la clase CustomDataset.
        Parámetros:
        - texts: Lista de textos.
        - labels: Lista de etiquetas correspondientes a los textos.
        - tokenizer: Objeto del tokenizador a utilizar.
        - max_length: Longitud máxima de la secuencia después de la tokenización.
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        """
        Devuelve la longitud del conjunto de datos.
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Obtiene un elemento del conjunto de datos.

        Parámetros:
        - idx: Índice del elemento a obtener.

        Devuelve:
        Un diccionario con 'input_ids', 'attention_mask' y 'labels'.
        """
        # Obtener el texto y la etiqueta del índice proporcionado
        text = str(self.texts[idx])
        label = int(self.labels[idx])

        # Tokenizar el texto
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        # Devolver el diccionario con los datos
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

Ahora, transformaremos nuestros datos al formato Datasets. Cada elemento del nuevo formato tendrá:
1. Inputs_ids: Identificadores numéricos de los tokens en el vocabulario del modelo.
2. attention_mask: Vector que indica a la red neuronal qué partes de la secuencia de entrada debe prestar atención y cuáles ignorar.
3. labels: La etiqueta asociada al texto

In [35]:
max_length = 128  # Puedes ajustar esto según tus necesidades

train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length)

## 4. Vectorización

## 5. Entrenamiento y evaluación de modelos


Para realizar el entrenamiento, cargamos el modelo en el entorno de ejecución.
Utilizaremos AutoModelForSequenceClassification(), donde carga modelos que van a ser ajustados para clasificar secuencias de textos.

Especificaremos el número de etiquetas que tenemos en nuestro problema de clasificación e introducimos unos diccionarios para gestionar y comprender mejor la salida del modelo.

In [36]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "NO_CYBERBULLYING", 1: "CYBERBULLYING"}
label2id = {"NO_CYBERBULLYING": 0, "CYBERBULLYING": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_name,  num_labels=2, id2label=id2label, label2id=label2id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-2019-90m and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Consecutivamente, definimos algunos parámetros para llevar a cabo el entrenamiento.

In [38]:
import accelerate

training_args = TrainingArguments(
    output_dir="modelo_test",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False
)

Generamos una función de cálculo de métricas de evaluación durante el entrenamiento:

In [39]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
f1_score = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy_value = accuracy.compute(predictions=predictions, references=labels)
    f1_score_value = f1_score.compute(predictions=predictions, references=labels)

    return {
        "accuracy": accuracy_value,
        "f1_score": f1_score_value,
    }

Creamos el objeto Trainer(), que nos permitirá llevar a cabo el ajuste del modelo:

In [40]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [41]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Score
1,0.278900,0.251259,{'accuracy': 0.9001956947162426},{'f1': 0.9411861614497529}
2,0.235700,0.300008,{'accuracy': 0.899077439194856},{'f1': 0.9413294327970096}
3,0.201500,0.453705,{'accuracy': 0.8969807100922561},{'f1': 0.9396643471142039}
4,0.164000,0.568260,{'accuracy': 0.8927872518870562},{'f1': 0.9369398996957987}


Trainer is attempting to log a value of "{'accuracy': 0.9001956947162426}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9411861614497529}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.899077439194856}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9413294327970096}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8969807100922561}" of type <class

TrainOutput(global_step=14308, training_loss=0.21605671068660234, metrics={'train_runtime': 3313.6413, 'train_samples_per_second': 34.542, 'train_steps_per_second': 4.318, 'total_flos': 7528922849126400.0, 'train_loss': 0.21605671068660234, 'epoch': 4.0})

## Evaluación

Ya entrenado el modelo, llevaremos a cabo su evaluación.

In [42]:
# Make predictions on the test data
trainer.evaluate(test_dataset)

Trainer is attempting to log a value of "{'accuracy': 0.8936509267801728}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.9374012638230649}" of type <class 'dict'> for key "eval/f1_score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.2630043625831604,
 'eval_accuracy': {'accuracy': 0.8936509267801728},
 'eval_f1_score': {'f1': 0.9374012638230649},
 'eval_runtime': 88.009,
 'eval_samples_per_second': 135.475,
 'eval_steps_per_second': 16.941,
 'epoch': 4.0}

Haremos un classification report para ver los scores relativos a cada categoría. Para eso, predeciremos las etiquetas sobre el test set con el método .predict(). Y obtendremos la etiqueta de cada predicción.

In [43]:
# Make predictions on the test data
predictions = trainer.predict(test_dataset)

In [44]:
predictions[0][0]

array([-2.0583556,  2.5988815], dtype=float32)

In [45]:
# Access the predicted labels
y_pred = predictions.predictions.argmax(axis=1)


Cogemos las etiquetas verdaderas y calculamos el classification report:

In [46]:
y_true = [x["labels"].item() for x in test_dataset]

In [47]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred))

[[1161  825]
 [ 443 9494]]
              precision    recall  f1-score   support

           0       0.72      0.58      0.65      1986
           1       0.92      0.96      0.94      9937

    accuracy                           0.89     11923
   macro avg       0.82      0.77      0.79     11923
weighted avg       0.89      0.89      0.89     11923



Podemos comparar los resultados con el clasificador obtenido mediante ingeniería de características.

Podemos predecir cualquier frase o tweet de forma simple:

In [48]:
# Create a function to predict for a single document
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def predict_single_document(document, max_length=128):
    # Tokenize the input document
    inputs = tokenizer(document, return_tensors="pt", max_length=max_length, truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move tensors to the same device as the model

    # Make the prediction
    with torch.no_grad():
      model_output = trainer.model(**inputs)

    # Access the predicted labels
    predicted_label = torch.argmax(model_output.logits).item()

    return predicted_label

Probamos con insertar tweets reales de cyberbullying y no cyberbullying, para saber si los predice correctamente.

In [49]:

input_text1 = 'Going to Africa. Hope I do not get AIDS. Just kidding. I am white'
input_text2 = 'I feel so lucky with my job and my friends.'

pred1 = predict_single_document(input_text1)
pred2 = predict_single_document(input_text2)

# label_mapping = {0:"no cyberbullying",1:"cyberbullyng"}

print("Clase texto 1: {}".format(pred1))
print("Clase texto 2: {}".format(pred2))

Clase texto 1: 1
Clase texto 2: 0


## **Comparación Ejercicio 1 y Ejercicio 2.**

Como podemos observar, la diferencia del haber usado un *algoritmo de aprendizaje supervisado de clasificación*, en nuestro caso Random Forest y de un *modelo de lenguaje* que ha sido previamente preentrenado donde ha realizado el fine-tunning de clasificación de tweets sobre cyberbullying, creando una red neuronal simple, es notorio en los resultados.

Obtenemos un accurracy  mayor a la hora de predecir tweets con el modelo de lenguaje (0.89) que con el algoritmo de clasificación (0.85).

Finalmente, es importante añadir que en cuanto a elección del modelo, elegiríamos el modelo de lenguaje ya que además de obtener un mayor porcentaje de predicción, es un modelo capaz de capturar la información contextual del texto.


